<a href="https://colab.research.google.com/github/isralennon/MSAAI530/blob/YAS/Best_Attempt_3_to_remove_Potential_Data_Leakage_in_Feature_Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://archive.ics.uci.edu/static/public/780/har70.zip
!unzip -q har70.zip

--2025-02-20 10:45:20--  https://archive.ics.uci.edu/static/public/780/har70.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘har70.zip’

har70.zip               [      <=>           ]  42.22M  1.11MB/s    in 38s     

2025-02-20 10:46:09 (1.10 MB/s) - ‘har70.zip’ saved [44267192]



In [4]:
import glob
import os
import numpy as np
import pandas as pd
from joblib import Parallel, delayed
from scipy.signal import hilbert, find_peaks, welch
from scipy import stats
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import GroupKFold, train_test_split, cross_validate
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm


class HAR70AdvancedAnalysis:
    """Handles specialized computations like phase synchronization & gait analysis."""

    def __init__(self, fs=50.0):
        self.fs = fs

    def compute_phase_sync(self, window_data):
        """Compute phase synchronization using only the first 75% of the window."""
        valid_part = int(len(window_data) * 0.75)  # Avoid future data
        return {
            f'phase_sync_{axis}': np.abs(np.mean(np.exp(1j * (
                np.angle(hilbert(window_data[f'back_{axis}'][:valid_part])) -
                np.angle(hilbert(window_data[f'thigh_{axis}'][:valid_part]))
            ))))
            for axis in 'xyz'
        }

    def analyze_gait(self, window_data):
        """Analyze gait parameters for walking activities using only past data."""
        valid_part = int(len(window_data) * 0.75)  # Avoid future data
        peaks, _ = find_peaks(window_data['back_x'].values[:valid_part], distance=20)
        if len(peaks) > 1:
            step_times = np.diff(peaks) / self.fs
            return {
                'step_time_mean': np.mean(step_times),
                'step_time_cv': np.std(step_times) / (np.mean(step_times) + 1e-10),
                'cadence': 60 / (np.mean(step_times) + 1e-10)
            }
        return {'step_time_mean': np.nan, 'step_time_cv': np.nan, 'cadence': np.nan}


class WindowGenerator:
    """Handles windowing of time series data efficiently."""

    def __init__(self, window_size, overlap):
        self.window_size = window_size
        self.step = int(window_size * (1 - overlap))

    def generate(self, data):
        """Generate sliding windows for each subject."""
        windows = []
        for _, subject_data in data.groupby('subject_id'):
            n_windows = (len(subject_data) - self.window_size) // self.step + 1
            for i in range(n_windows):
                start_idx = i * self.step
                end_idx = start_idx + self.window_size
                if end_idx <= len(subject_data):
                    windows.append(subject_data.iloc[start_idx:end_idx])
        return windows


class HAR70Analyzer:
    """Main class for processing the HAR70+ dataset and training models."""

    def __init__(self, data_dir='har70plus', n_jobs=-1, window_size=256, overlap=0.5):
        self.data_dir = data_dir
        self.n_jobs = n_jobs
        self.window_generator = WindowGenerator(window_size, overlap)
        self.advanced = HAR70AdvancedAnalysis()
        self.scaler = None
        self.classifier = None
        self.feature_columns = None
        self.fs = 50

    def load_data(self):
        """Load and validate HAR70+ dataset."""
        files = glob.glob(os.path.join(self.data_dir, '*.csv'))
        all_data = [pd.read_csv(f).assign(subject_id=os.path.basename(f).split('.')[0]) for f in tqdm(files, desc='Loading data')]

        data = pd.concat(all_data, ignore_index=True)
        data.dropna(inplace=True)
        return data

    def _extract_features(self, window_data):
        """Extracts features from a single window while preventing data leakage."""
        valid_part = int(len(window_data) * 0.75)  # Ensure no future data usage
        features = {
            'subject_id': window_data['subject_id'].iloc[0]
        }

        for col in ['back_x', 'back_y', 'back_z', 'thigh_x', 'thigh_y', 'thigh_z']:
            sig = window_data[col].values[:valid_part]  # Use only first 75%

            # Time-domain features
            features.update({
                f'{col}_{stat}': func(sig)
                for stat, func in {
                    'mean': np.mean,
                    'std': np.std,
                    'rms': lambda x: np.sqrt(np.mean(x**2)),
                    'max': lambda x: np.max(np.abs(x)),
                    'kurtosis': stats.kurtosis,
                    'skewness': stats.skew
                }.items()
            })

            # Frequency-domain features
            f, psd = welch(sig, fs=self.fs, nperseg=min(len(sig), 256))
            total_power = np.sum(psd)
            psd_norm = psd / (total_power + 1e-10)

            features.update({
                f'{col}_dom_freq': f[np.argmax(psd)],
                f'{col}_spec_entropy': -np.sum(psd_norm * np.log2(psd_norm + 1e-10)),
                f'{col}_spec_centroid': np.sum(f * psd) / (total_power + 1e-10)
            })

            features.update({
                f'{col}_power_{band}': np.sum(psd[idx]) / (total_power + 1e-10)
                for band, idx in {
                    'low': f < 1.0,
                    'mid': (f >= 1.0) & (f < 3.0),
                    'high': f >= 3.0
                }.items()
            })

        # Compute phase sync and gait analysis only using past data
        features.update(self.advanced.compute_phase_sync(window_data.iloc[:valid_part]))
        if stats.mode(window_data['label'].values[:valid_part], keepdims=False)[0] in [1, 3]:
            features.update(self.advanced.analyze_gait(window_data.iloc[:valid_part]))

        # Assign the label based on the **first 75% of the window** to prevent future influence
        features['label'] = stats.mode(window_data['label'].values[:valid_part], keepdims=False)[0]

        return features

    def extract_features(self, data):
        """Extract features using parallel processing."""
        windows = self.window_generator.generate(data)
        features_list = Parallel(n_jobs=self.n_jobs)(
            delayed(self._extract_features)(window) for window in tqdm(windows, desc='Extracting features')
        )

        features_df = pd.DataFrame(features_list)
        self.feature_columns = [col for col in features_df.columns if col not in ['label', 'subject_id']]
        return features_df

    def train_model(self, data, cv_folds=5):
        """Train and evaluate classifier with proper data leakage prevention."""

        # Split by subject (NO SUBJECT OVERLAP)
        subjects = data['subject_id'].unique()
        train_subjects, test_subjects = train_test_split(subjects, test_size=0.2, random_state=42)

        train_data = data[data['subject_id'].isin(train_subjects)]
        test_data = data[data['subject_id'].isin(test_subjects)]

        # Extract features separately
        train_features = self.extract_features(train_data)
        test_features = self.extract_features(test_data)

        X_train = train_features[self.feature_columns]
        y_train = train_features['label']
        groups_train = train_features['subject_id']

        X_test = test_features[self.feature_columns]
        y_test = test_features['label']

        # Scale features (NO SCALING BEFORE SPLIT)
        self.scaler = StandardScaler()
        X_train_scaled = self.scaler.fit_transform(X_train)
        X_test_scaled = self.scaler.transform(X_test)

        # Train model using `GroupKFold`
        self.classifier = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=self.n_jobs)
        cv = GroupKFold(n_splits=cv_folds)

        cross_validate(self.classifier, X_train_scaled, y_train, cv=cv, groups=groups_train, n_jobs=self.n_jobs)

        self.classifier.fit(X_train_scaled, y_train)
        y_pred = self.classifier.predict(X_test_scaled)

        return classification_report(y_test, y_pred, zero_division=0)


if __name__ == "__main__":
    analyzer = HAR70Analyzer()
    data = analyzer.load_data()
    print(analyzer.train_model(data))

Extracting features: 100%|██████████| 3978/3978 [00:52<00:00, 76.19it/s]


              precision    recall  f1-score   support

           1       0.97      1.00      0.98      1974
           3       0.90      0.21      0.35        89
           4       0.75      0.60      0.67         5
           5       0.57      0.57      0.57         7
           6       1.00      1.00      1.00       867
           7       0.99      1.00      0.99       702
           8       1.00      0.97      0.99       334

    accuracy                           0.98      3978
   macro avg       0.88      0.76      0.79      3978
weighted avg       0.98      0.98      0.97      3978



# Diagnostic code from ChatGPT 4o  to identify potential data leakage:

1. Check for Label Consistency in Windows
We verify whether stats.mode(window_data['label'].values[:valid_part]) matches the final label in the window (window_data['label'].values[-1]).
If they differ frequently, future information might be indirectly leaking into feature extraction.
2. Validate Welch’s PSD Calculation
Ensure nperseg in welch() does not exceed valid_part.
If nperseg > valid_part, then Welch’s method is using future data.
3. Confirm Peak Detection Uses Only Past Data
Ensure that detected peaks are only from valid_part and that no peaks from the future (beyond valid_part) influence calculations.


In [5]:
import numpy as np
import pandas as pd
from scipy.signal import welch, find_peaks
from scipy import stats

def check_label_consistency(data, window_size=256, valid_ratio=0.75):
    """Check if future labels are leaking into feature extraction."""
    valid_part = int(window_size * valid_ratio)
    inconsistent_labels = 0
    total_windows = 0

    for _, subject_data in data.groupby('subject_id'):
        n_windows = (len(subject_data) - window_size) // (int(window_size * (1 - 0.5))) + 1
        for i in range(n_windows):
            start_idx = i * int(window_size * (1 - 0.5))
            end_idx = start_idx + window_size
            if end_idx > len(subject_data):
                continue

            window_labels = subject_data['label'].values[start_idx:end_idx]
            mode_label = stats.mode(window_labels[:valid_part], keepdims=False)[0]
            future_label = window_labels[-1]

            if mode_label != future_label:
                inconsistent_labels += 1

            total_windows += 1

    print(f"Label Inconsistency Rate: {inconsistent_labels / total_windows:.2%} ({inconsistent_labels}/{total_windows})")

def check_welch_nperseg(data, window_size=256, valid_ratio=0.75, fs=50):
    """Ensure Welch's nperseg does not exceed valid_part of the window."""
    valid_part = int(window_size * valid_ratio)
    violations = 0
    total_checks = 0

    for _, subject_data in data.groupby('subject_id'):
        n_windows = (len(subject_data) - window_size) // (int(window_size * (1 - 0.5))) + 1
        for i in range(n_windows):
            start_idx = i * int(window_size * (1 - 0.5))
            end_idx = start_idx + window_size
            if end_idx > len(subject_data):
                continue

            sig = subject_data['back_x'].values[start_idx:start_idx+valid_part]  # Only past data
            nperseg = min(len(sig), 256)

            if nperseg > valid_part:
                violations += 1

            total_checks += 1

    print(f"Welch's nperseg Violation Rate: {violations / total_checks:.2%} ({violations}/{total_checks})")

def check_peak_detection(data, window_size=256, valid_ratio=0.75, fs=50):
    """Ensure peak detection only uses past data."""
    valid_part = int(window_size * valid_ratio)
    peak_violations = 0
    total_checks = 0

    for _, subject_data in data.groupby('subject_id'):
        n_windows = (len(subject_data) - window_size) // (int(window_size * (1 - 0.5))) + 1
        for i in range(n_windows):
            start_idx = i * int(window_size * (1 - 0.5))
            end_idx = start_idx + window_size
            if end_idx > len(subject_data):
                continue

            sig = subject_data['back_x'].values[start_idx:start_idx+valid_part]  # Only past data
            peaks, _ = find_peaks(sig, distance=20)

            if len(peaks) > 0 and np.any(peaks >= valid_part):
                peak_violations += 1  # Peaks detected beyond valid range

            total_checks += 1

    print(f"Peak Detection Violation Rate: {peak_violations / total_checks:.2%} ({peak_violations}/{total_checks})")

# Load your dataset
analyzer = HAR70Analyzer()
data = analyzer.load_data()

# Run the checks
check_label_consistency(data)
check_welch_nperseg(data)
check_peak_detection(data)


Loading data: 100%|██████████| 18/18 [00:04<00:00,  3.75it/s]


Label Inconsistency Rate: 11.52% (2030/17627)
Welch's nperseg Violation Rate: 0.00% (0/17627)
Peak Detection Violation Rate: 0.00% (0/17627)
